In [3]:
import pandas as pd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import xgboost as xgb
from sklearn.metrics import mean_squared_error


#####imputing, ncoidng etc


In [ ]:
df = pd.read_csv("/content/Electric_Consumption_And_Cost__2010_-_Feb_2025__20250314.csv",low_memory=False)

In [ ]:

df.shape

(521035, 27)

In [ ]:
df['Revenue Date'] = pd.to_datetime(df['Revenue Month'], errors='coerce')

# Extract year into a new column
df['Revenue Year'] = df['Revenue Date'].dt.year


filtered_df =  (df[df['Revenue Year'] == 2021])




In [ ]:
filtered_df.shape

(41367, 29)

In [ ]:
pd.set_option('display.max_columns', None)
pd.reset_option('display.width')


In [ ]:
df_temp = df[df['Borough'].str.upper() == 'STATEN ISLAND']

# Remove February 2024 based on 'Revenue Month'
# Direct string comparison without datetime parsing
df_temp = df_temp[df_temp['Revenue Month'] != '2019-02']


In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
import plotly.graph_objects as go
from prophet import Prophet

In [ ]:
kw_df = df_temp[['Revenue Month', 'Consumption (KW)']].dropna()
kw_df = kw_df.groupby('Revenue Month').sum().reset_index()
kw_df = kw_df.sort_values('Revenue Month')

# Prepare data for Prophet
prophet_df = kw_df.rename(columns={'Revenue Month': 'ds', 'Consumption (KW)': 'y'})
prophet_df['floor'] = 0
prophet_df['y'] = np.log1p(prophet_df['y'])  # Log-transform

# Train model
model = Prophet(growth='linear', seasonality_mode='multiplicative',changepoint_prior_scale=0.01 )
model.add_seasonality(name='monthly', period=30.5, fourier_order=5)
model.fit(prophet_df)

# Create future dataframe (24 months ahead)
future = model.make_future_dataframe(periods=24, freq='MS')
future['floor'] = 0
forecast = model.predict(future)

# Evaluate performance
forecast_actual = forecast[forecast['ds'].isin(prophet_df['ds'])]
rmse = mean_squared_error(prophet_df['y'], forecast_actual['yhat'])
print("Prophet RMSE (log scale):", rmse)

# Inverse transform
prophet_df['y'] = np.expm1(prophet_df['y'])
forecast['yhat'] = np.expm1(forecast['yhat'])
forecast['yhat_upper'] = np.expm1(forecast['yhat_upper'])
forecast['yhat_lower'] = np.expm1(forecast['yhat_lower'])


# Get cutoff date for splitting actual vs future
cutoff = prophet_df['ds'].max()
forecast_future = forecast[forecast['ds'] > cutoff]
forecast_full = forecast[forecast['ds'] <= cutoff]

# Compute % growth over forecast period
start_val = forecast_future.iloc[0]['yhat']
end_val = forecast_future.iloc[-1]['yhat']
growth_pct = ((end_val - start_val) / start_val) * 100

# Plot
fig = go.Figure()

# Actual historical values
fig.add_trace(go.Scatter(
    x=prophet_df['ds'], y=prophet_df['y'],
    mode='lines', name='Actual',
    line=dict(color='blue', width=2,shape='spline')
))

# Forecast (future only)
fig.add_trace(go.Scatter(
    x=forecast_future['ds'], y=forecast_future['yhat'],
    mode='lines', name='Forecast',
    line=dict(color='blue', width=3, shape='spline')
))

# Confidence Interval (CI) band
fig.add_trace(go.Scatter(
    x=forecast_future['ds'],
    y=forecast_future['yhat_upper'],
    mode='lines',
    line=dict(width=0),
    showlegend=False
))
fig.add_trace(go.Scatter(
    x=forecast_future['ds'],
    y=forecast_future['yhat_lower'],
    mode='lines',
    fill='tonexty',
    fillcolor='rgba(0,100,80,0.2)',
    line=dict(width=0),
    name='Confidence Interval',
    showlegend=True
))
start_date = forecast_future['ds'].iloc[0]
start_y = forecast_future['yhat_upper'].iloc[0]

# Add % growth text box
fig.add_annotation(
    text=f"Projected KW Growth (Next 2 Years): <b>{growth_pct:.2f}%</b>",
    xref="paper", yref="paper",
    x=1,
    y=.8,
    showarrow=False,

    font=dict(size=16, color="black"),
    align="center",
    bordercolor="black",
    borderwidth=1,
    borderpad=10,
    bgcolor="white",
    opacity=0.95
)

# Clean layout
fig.update_layout(
    title="📈 KW Forecast Using Prophet (Next 2 Years)",
    xaxis_title="Month",
    yaxis_title="KW Usage",
    template="simple_white",
    font=dict(size=14),
    legend=dict(x=0.01, y=0.99),
    hovermode="x unified",
    margin=dict(t=100)
)

fig.show()


INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpbtxusu5v/oy9zt9q1.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpbtxusu5v/hkwa5c1e.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=18495', 'data', 'file=/tmp/tmpbtxusu5v/oy9zt9q1.json', 'init=/tmp/tmpbtxusu5v/hkwa5c1e.json', 'output', 'file=/tmp/tmpbtxusu5v/prophet_model2tbqr8qu/prophet_model-20250429124052.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
12:40:52 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
12:40:52 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
<ipython-input-6-aabe3395a981>:21: Futur

Prophet RMSE (log scale): 0.09636236893027798


In [ ]:
#!pip install jupyter_bokeh

import panel as pn
pn.extension('plotly')


In [ ]:
# Wrap the Plotly figure in a Panel pane
plot_pane = pn.pane.Plotly(fig, config={'responsive': True})

# Display it
plot_pane.show()


Launching server at http://localhost:39273


In [ ]:
import panel as pn
pn.extension('plotly')

pn.Column("# My Panel", pn.pane.Plotly(fig))


Column
    [0] Markdown(str)
    [1] Plotly(Figure)

In [ ]:
import pickle

# Save the trained model
with open('prophet_model.pkl', 'wb') as f:
    pickle.dump(model, f)

# Save the forecasted results
forecast.to_csv('forecast_results.csv', index=False)

# Save the actual data
prophet_df.to_csv('actual_data.csv', index=False)


In [ ]:
import pickle

# Save the trained Prophet model
with open('prophet_model.pkl', 'wb') as f:
    pickle.dump(model, f)

# Save forecast and prophet_df as CSVs
forecast.to_csv('forecast.csv', index=False)
prophet_df.to_csv('actual_data.csv', index=False)


In [1]:
import pandas as pd
import pickle

# Load model
with open('prophet_model.pkl', 'rb') as f:
    model = pickle.load(f)

# Load data
forecast = pd.read_csv('forecast.csv')
prophet_df = pd.read_csv('actual_data.csv')

"""
# OPTIONAL: Convert 'ds' column to datetime if needed
forecast['ds'] = pd.to_datetime(forecast['ds'])
prophet_df['ds'] = pd.to_datetime(prophet_df['ds'])
"""

"\n# OPTIONAL: Convert 'ds' column to datetime if needed\nforecast['ds'] = pd.to_datetime(forecast['ds'])\nprophet_df['ds'] = pd.to_datetime(prophet_df['ds'])\n"

In [4]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
import plotly.graph_objects as go
from prophet import Prophet

# Get cutoff date for splitting actual vs future
cutoff = prophet_df['ds'].max()
forecast_future = forecast[forecast['ds'] > cutoff]
forecast_full = forecast[forecast['ds'] <= cutoff]

# Compute % growth over forecast period
start_val = forecast_future.iloc[0]['yhat']
end_val = forecast_future.iloc[-1]['yhat']
growth_pct = ((end_val - start_val) / start_val) * 100

# Plot
fig = go.Figure()

# Actual historical values
fig.add_trace(go.Scatter(
    x=prophet_df['ds'], y=prophet_df['y'],
    mode='lines', name='Actual',
    line=dict(color='blue', width=2,shape='spline')
))

# Forecast (future only)
fig.add_trace(go.Scatter(
    x=forecast_future['ds'], y=forecast_future['yhat'],
    mode='lines', name='Forecast',
    line=dict(color='blue', width=3, shape='spline')
))

# Confidence Interval (CI) band
fig.add_trace(go.Scatter(
    x=forecast_future['ds'],
    y=forecast_future['yhat_upper'],
    mode='lines',
    name = 'Upper Bound',
    line=dict(width=0),
    showlegend=False,

))
fig.add_trace(go.Scatter(
    x=forecast_future['ds'],
    y=forecast_future['yhat_lower'],
    mode='lines',
    fill='tonexty',
    fillcolor='rgba(0,100,80,0.2)',
    line=dict(width=0),
    name='Lower Bound',
    showlegend=True,

))
start_date = forecast_future['ds'].iloc[0]
start_y = forecast_future['yhat_upper'].iloc[0]

# Add % growth text box
fig.add_annotation(
    text=f"Projected KW Growth (Next 2 Years): <b>{growth_pct:.2f}%</b>",
    xref="paper", yref="paper",
    x=1,
    y=.8,
    showarrow=False,

    font=dict(size=16, color="black"),
    align="center",
    bordercolor="black",
    borderwidth=1,
    borderpad=10,
    bgcolor="white",
    opacity=0.95
)

# Clean layout
fig.update_layout(
    title="Forecast of KW Usage in Staten Island for the next 2 years",
    xaxis_title="Month",
    yaxis_title="KW Usage",
    template="simple_white",
    font=dict(size=14),
    legend=dict(x=0.01, y=0.99),
    hovermode="x unified",
    margin=dict(t=100)
)

fig.show()
